In [ ]:
# Step 1: Import Required Libraries
import os
import nibabel as nib
import numpy as np
from sklearn.model_selection import train_test_split
from monai.transforms import Compose, EnsureChannelFirst, ScaleIntensity, Resize

# Step 2: Define the Original Volume File Directory
volume_dir = "dataset/segmentations/images/images"  # Replace with your volume files folder path
new_image_dir = "input"  # Path to save extracted images
new_mask_dir = "input"  # Path to save generated masks

# Create directories if they don't exist
os.makedirs(new_image_dir, exist_ok=True)
os.makedirs(new_mask_dir, exist_ok=True)

# Step 3: Process Volume Files
volume_files = sorted([os.path.join(volume_dir, f) for f in os.listdir(volume_dir) if f.endswith(".nii")])

# Loop through volume files and create images and masks
for idx, volume_file in enumerate(volume_files):
    # Load the volume file
    volume = nib.load(volume_file).get_fdata()

    # Extract an image slice (e.g., middle slice)
    mid_slice_idx = volume.shape[-1] // 2
    image_slice = volume[:, :, mid_slice_idx]

    # Create a synthetic mask (e.g., threshold-based segmentation)
    mask_slice = (image_slice > np.mean(image_slice)).astype(np.uint8)

    # Save the image and mask
    image_path = os.path.join(new_image_dir, f"image_{idx:03d}.nii")
    mask_path = os.path.join(new_mask_dir, f"mask_{idx:03d}.nii")
    nib.save(nib.Nifti1Image(image_slice, affine=np.eye(4)), image_path)
    nib.save(nib.Nifti1Image(mask_slice, affine=np.eye(4)), mask_path)

print(f"Processed {len(volume_files)} volumes into images and masks.")
print(f"New image directory: {new_image_dir}")
print(f"New mask directory: {new_mask_dir}")

# Step 4: Collect and Sort Paths
image_paths = sorted([os.path.join(new_image_dir, f) for f in os.listdir(new_image_dir) if f.endswith(".nii")])
mask_paths = sorted([os.path.join(new_mask_dir, f) for f in os.listdir(new_mask_dir) if f.endswith(".nii")])

# Validate paths
if not image_paths:
    raise ValueError("No image files found in the newly created directory.")
if not mask_paths:
    raise ValueError("No mask files found in the newly created directory.")
if len(image_paths) != len(mask_paths):
    raise ValueError(
        f"Mismatch between the number of images ({len(image_paths)}) and masks ({len(mask_paths)})."
    )

# Step 5: Split Data



Processed 131 volumes into images and masks.
New image directory: input
New mask directory: input
Training images: 209, Validation images: 53
Training masks: 209, Validation masks: 53
Example training image path: input\mask_048.nii
Example training mask path: input\mask_048.nii


ValueError: All arrays must be of the same length

In [12]:
image_paths = 'input/image'
mask_paths = 'input/mask'

train_images, val_images, train_masks, val_masks = train_test_split(
    image_paths, mask_paths, test_size=0.2, random_state=42
)

# Step 6: Define Transforms
transform = Compose([
    EnsureChannelFirst(),
    ScaleIntensity(),
    Resize((128, 128, 64))
])

# Step 7: Debug Data
print(f"Training images: {len(train_images)}, Validation images: {len(val_images)}")
print(f"Training masks: {len(train_masks)}, Validation masks: {len(val_masks)}")
print("Example training image path:", train_images[0])
print("Example training mask path:", train_masks[0])

# Step 8: Save Data Splits (Optional)
import pandas as pd

data_splits = pd.DataFrame({
    "train_images": train_images,
    "train_masks": train_masks,
    "val_images": val_images,
    "val_masks": val_masks,
})

data_splits.to_csv("data_splits.csv", index=False)
print("Data splits saved to 'data_splits.csv'.")

# Step 9: Prepare Data for Model Training
from torch.utils.data import Dataset

class NiiDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = nib.load(self.image_paths[idx]).get_fdata()
        mask = nib.load(self.mask_paths[idx]).get_fdata()

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

# Initialize Dataset
train_dataset = NiiDataset(train_images, train_masks, transform=transform)
val_dataset = NiiDataset(val_images, val_masks, transform=transform)

print(f"Number of samples in training dataset: {len(train_dataset)}")
print(f"Number of samples in validation dataset: {len(val_dataset)}")

# Debug a Sample
sample_image, sample_mask = train_dataset[0]
print("Sample image shape (dataset):", sample_image.shape)
print("Sample mask shape (dataset):", sample_mask.shape)

ValueError: Found input variables with inconsistent numbers of samples: [11, 10]

In [14]:
import os
import nibabel as nib
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import Compose, ToTensor, Normalize
import matplotlib.pyplot as plt
from monai.transforms import Spacing, ScaleIntensity, Resize, AddChannel
from monai.networks.nets import UNet
from monai.metrics import DiceMetric
from sklearn.model_selection import train_test_split


ImportError: cannot import name 'AddChannel' from 'monai.transforms' (c:\Users\piyus\AppData\Local\Programs\Python\Python311\Lib\site-packages\monai\transforms\__init__.py)

In [15]:
class LiverTumorDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = nib.load(self.image_paths[idx]).get_fdata()
        mask = nib.load(self.mask_paths[idx]).get_fdata()

        # Normalize
        image = (image - np.min(image)) / (np.max(image) - np.min(image))

        # Apply transformations
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return torch.tensor(image, dtype=torch.float32), torch.tensor(mask, dtype=torch.float32)


In [19]:
import numpy as np
from monai.transforms import Compose, ScaleIntensity, Resize
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# Custom AddChannel transform
class AddChannelCustom:
    def __call__(self, data):
        # Assuming data is a dictionary with 'image' and 'label' keys
        data['image'] = np.expand_dims(data['image'], axis=0)  # Add channel dimension to image
        data['label'] = np.expand_dims(data['label'], axis=0)  # Add channel dimension to label
        return data

# Paths to your .nii files
image_dir = "input/image"
mask_dir = "input/mask"

image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.nii')]
mask_paths = [os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith('.nii')]

# Split into train and validation
train_images, val_images, train_masks, val_masks = train_test_split(image_paths, mask_paths, test_size=0.2, random_state=42)

# Define transformations
transform = Compose([AddChannelCustom(), ScaleIntensity(), Resize((128, 128, 64))])

# Create datasets and dataloaders (Assuming you have a custom Dataset class)
train_dataset = LiverTumorDataset(train_images, train_masks, transform=transform)
val_dataset = LiverTumorDataset(val_images, val_masks, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)


In [21]:
from monai.networks.nets import UNet

model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to('cuda')


In [23]:
from monai.metrics import DiceMetric  # Add this import statement

# Define loss function, optimizer, and metric
loss_function = DiceLoss(sigmoid=True)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
dice_metric = DiceMetric(include_background=False, reduction="mean")


In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from monai.metrics import DiceMetric
from monai.networks.nets import UNet
from monai.data import Dataset
from monai.transforms import Compose, ToTensord

# Custom transform to add channel dimension
class AddChannelCustom:
    def __init__(self, keys):
        self.keys = keys
    
    def __call__(self, data):
        for key in self.keys:
            # Add an extra channel dimension to each key (image, label)
            if key in data:
                data[key] = np.expand_dims(data[key], axis=0)  # Add channel dimension (for 3D data)
        return data

# Dataset class
class LiverTumorDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform

    def __getitem__(self, idx):
        image = np.load(self.image_paths[idx])  # Replace with your image loading logic
        label = np.load(self.label_paths[idx])  # Replace with your label loading logic

        # Apply the transform (assuming the transform expects a dictionary)
        if self.transform:
            data = {'image': image, 'label': label}
            data = self.transform(data)  # Apply the transform
            return data['image'], data['label']  # Return image and label
        else:
            return image, label

    def __len__(self):
        return len(self.image_paths)

# Define transforms (you can add more transforms here if needed)
transform = Compose([
    AddChannelCustom(keys=['image', 'label']),  # Add channel dimension for both image and label
    ToTensord(keys=['image', 'label'])     # Convert image and label to tensors
])

# Prepare data loaders
train_image_paths = ['path_to_train_image1.npy', 'path_to_train_image2.npy']  # Replace with actual paths
train_label_paths = ['path_to_train_label1.npy', 'path_to_train_label2.npy']
val_image_paths = ['path_to_val_image1.npy', 'path_to_val_image2.npy']
val_label_paths = ['path_to_val_label1.npy', 'path_to_val_label2.npy']

train_dataset = LiverTumorDataset(train_image_paths, train_label_paths, transform)
val_dataset = LiverTumorDataset(val_image_paths, val_label_paths, transform)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

# Define the UNet model
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to('cuda')

# Define loss function and optimizer
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Define the Dice metric
dice_metric = DiceMetric(include_background=False, reduction="mean")

# Training loop
num_epochs = 50
best_metric = -1
best_metric_epoch = -1

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    model.train()
    epoch_loss = 0
    for batch_data in train_loader:
        inputs, labels = batch_data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch Loss: {epoch_loss/len(train_loader)}")

    # Validation
    model.eval()
    with torch.no_grad():
        val_dice = 0
        for val_data in val_loader:
            val_inputs, val_labels = val_data
            val_inputs, val_labels = val_inputs.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_inputs)
            dice_metric(y_pred=val_outputs, y=val_labels)
        metric = dice_metric.aggregate().item()
        dice_metric.reset()

        print(f"Validation Dice Score: {metric}")
        if metric > best_metric:
            best_metric = metric
            best_metric_epoch = epoch + 1
            torch.save(model.state_dict(), "best_metric_model.pth")
            print("Model Saved!")

print(f"Training complete. Best validation Dice score: {best_metric} at epoch {best_metric_epoch}")


Epoch 1/50


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_train_image2.npy'